In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from pandas.api.types import CategoricalDtype
import random
import implicit
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

from implicit.datasets.movielens import get_movielens
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.datasets.movielens import get_movielens
from implicit.lmf import LogisticMatrixFactorization
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)
import tqdm
import codecs

In [2]:
retail_data = pd.read_json('data/cus_user_quality_others.json') # This may take a couple minutes
# df2 = pd.read_csv('data/product_brand_type_color.csv') # This may take a couple minutes

In [3]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117162 entries, 0 to 117161
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   CUSTOMER_DIM_KEY  117162 non-null  int64  
 1   PRODUCT_DIM_KEY   117162 non-null  int64  
 2   QUANTITY          117161 non-null  float64
 3   PRODUCT_NAME      117155 non-null  object 
 4   COLOR             117162 non-null  object 
 5   PRODUCT_TYPE      117162 non-null  object 
 6   CATEGORY          117162 non-null  object 
 7   BRAND             117162 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 7.2+ MB


In [4]:
cleaned_retail = retail_data.loc[pd.isnull(retail_data.PRODUCT_NAME) == False]

cleaned_retail['QUANTITY'] = cleaned_retail['QUANTITY'].fillna(0.0)

print('Duplicated rows: ' + str(cleaned_retail.duplicated().sum()))
cleaned_retail.drop_duplicates(inplace=True)

G:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Duplicated rows: 7426


G:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
cleaned_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109729 entries, 0 to 117161
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   CUSTOMER_DIM_KEY  109729 non-null  int64  
 1   PRODUCT_DIM_KEY   109729 non-null  int64  
 2   QUANTITY          109729 non-null  float64
 3   PRODUCT_NAME      109729 non-null  object 
 4   COLOR             109729 non-null  object 
 5   PRODUCT_TYPE      109729 non-null  object 
 6   CATEGORY          109729 non-null  object 
 7   BRAND             109729 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 7.5+ MB


In [6]:
item_lookup = cleaned_retail[['PRODUCT_DIM_KEY', 'PRODUCT_NAME']].drop_duplicates() # Only get unique item/description pairs
#item_lookup['PRODUCT_DIM_KEY'] = item_lookup.PRODUCT_DIM_KEY.astype(str) # Encode as strings for future lookup ease

In [7]:
def threshold_likes(df, uid_min, mid_min, user_column = 'CUSTOMER_DIM_KEY', item_column = 'PRODUCT_DIM_KEY'):
    n_users = df[user_column].unique().shape[0]
    n_items = df[item_column].unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Starting likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    
    done = False
    while not done:
        starting_shape = df.shape[0]
        mid_counts = df.groupby(user_column)[item_column].count()
        df = df[~df[user_column].isin(mid_counts[mid_counts < mid_min].index.tolist())]
        uid_counts = df.groupby(item_column)[user_column].count()
        df = df[~df[item_column].isin(uid_counts[uid_counts < uid_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
    
    assert(df.groupby(user_column)[item_column].count().min() >= mid_min)
    assert(df.groupby(item_column)[user_column].count().min() >= uid_min)
    
    n_users = df[user_column].unique().shape[0]
    n_items = df[item_column].unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    return df

In [8]:
df_lim = threshold_likes(cleaned_retail, 2, 2)

Starting likes info
Number of users: 49603
Number of models: 40596
Sparsity: 0.005%
Ending likes info
Number of users: 15424
Number of models: 12385
Sparsity: 0.030%


In [9]:
def create_matrix(cleaned_retail, row_feature, col_feature):
    cleaned_retail[row_feature] = cleaned_retail[row_feature].astype(int) # Convert to int for customer ID
    cleaned_retail = cleaned_retail[[col_feature, 'QUANTITY', row_feature]] # Get rid of unnecessary info
    grouped_cleaned = cleaned_retail.groupby([row_feature, col_feature]).sum().reset_index() # Group together
    grouped_cleaned.QUANTITY.loc[grouped_cleaned.QUANTITY == 0] = 1 # Replace a sum of zero purchases with a one to
    # indicate purchased
    grouped_purchased = grouped_cleaned.query('QUANTITY > 0') # Only get customers where purchase totals were positive
    
    customers = list(np.sort(grouped_purchased[row_feature].unique())) # Get our unique customers
    products = list(grouped_purchased[col_feature].unique()) # Get our unique products that were purchased
    quantity = list(grouped_purchased.QUANTITY) # All of our purchases

    rows = grouped_purchased[row_feature].astype('category', CategoricalDtype(categories = customers)).cat.codes 
    # Get the associated row indices
    cols = grouped_purchased[col_feature].astype('category', CategoricalDtype(categories = products)).cat.codes 
    # Get the associated column indices
    purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))
    return purchases_sparse

In [33]:
cleaned_retail['CUSTOMER_DIM_KEY'] = cleaned_retail['CUSTOMER_DIM_KEY'].astype(int) # Convert to int for customer ID
cleaned_retail = cleaned_retail[['PRODUCT_DIM_KEY', 'QUANTITY', 'CUSTOMER_DIM_KEY']] # Get rid of unnecessary info
grouped_cleaned = cleaned_retail.groupby(['CUSTOMER_DIM_KEY', 'PRODUCT_DIM_KEY']).sum().reset_index() # Group together
grouped_cleaned.QUANTITY.loc[grouped_cleaned.QUANTITY == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
grouped_purchased = grouped_cleaned.query('QUANTITY > 0') # Only get customers where purchase totals were positive

customers = list(np.sort(grouped_purchased['CUSTOMER_DIM_KEY'].unique())) # Get our unique customers
products = list(grouped_purchased['PRODUCT_DIM_KEY'].unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.QUANTITY) # All of our purchases

rows = grouped_purchased.CUSTOMER_DIM_KEY.astype('category', CategoricalDtype(categories = customers)).cat.codes 
# Get the associated row indices
cols = grouped_purchased.PRODUCT_DIM_KEY.astype('category', CategoricalDtype(categories = products)).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

In [35]:
purchases_sparse = create_matrix(df_lim, 'CUSTOMER_DIM_KEY', 'PRODUCT_DIM_KEY')

In [36]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered

In [37]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

In [152]:
import time
def train_model(ratings, model_name, factors =20, regularization=0.1,alpha = 10):
    # lets weight these models by bm25weight.
    print("weighting matrix by bm25_weight")
    ratings = (bm25_weight(ratings, B=0.9) * 5).tocsr()
    if model_name == "als":
        model = AlternatingLeastSquares(factors =factors, regularization=regularization)

    elif model_name == "bpr":
        model = BayesianPersonalizedRanking()

    elif model_name == "lmf":
        model = LogisticMatrixFactorization()

    elif model_name == "tfidf":
        model = TFIDFRecommender()

    elif model_name == "cosine":
        model = CosineRecommender()

    elif model_name == "bm25":
        model = BM25Recommender(B=0.2)

    else:
        raise NotImplementedError("TODO: model %s" % model_name)
    # train the model
    print("training model %s", model_name)
    start = time.time()
    model.fit(ratings*alpha)
    print("trained model '%s' in %s", model_name, time.time() - start)
        
    item_vecs = model.user_factors
    user_vecs = model.item_factors
    return item_vecs, user_vecs, model

In [39]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr) 

In [40]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [107]:
item_vecs, user_vecs, model = train_model(product_train, 'als')
param_grid['num_factors']

weighting matrix by bm25_weight
training model %s als



trained model '%s' in %s als 0.5285060405731201


[10, 20, 40, 80, 120]

In [125]:
'''
param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'regularization': [0.0, 1e-5, 1e-3, 1e-1, 1e1, 1e2],
              'alpha': [1, 10, 50, 100, 500, 1000]}
'''

param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'regularization': [0.0, 1e-5, 1e-3, 1e-1, 1e1, 1e2],
              'alpha': [1, 10, 50, 100, 500, 1000]}

def cross_validation(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    training_set = (bm25_weight(product_train, B=0.9) * 5).tocsr()
    if model_name == "als":
        model = AlternatingLeastSquares()
        
    best_auc = 0
    best_factors = 0
    best_reg = 0
    best_alpha = 0
    for factor in param_grid['num_factors']:
        for regu in param_grid['regularization']:
            for alpha in param_grid['alpha']:
                model = AlternatingLeastSquares(factors = factor, regularization = regu)
                model.fit(training_set*alpha)
                item_vecs = model.user_factors
                user_vecs = model.item_factors
                store_auc, pop_auc = calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
                print(factor, regu, alpha, store_auc, pop_auc)
                
                if store_auc > best_auc:
                    best_auc = store_auc
                    best_factors = factor
                    best_reg = regu
                    best_alpha = alpha
    print(best_auc, best_factors, best_reg, best_alpha)
    return best_auc, best_factors, best_reg, best_alpha
    # print("trained model '%s' in %s", model_name, time.time() - start)
        
    


In [126]:
cross_validation(purchases_sparse, 'als', param_grid)


10 0.0 1 0.792 0.749



10 0.0 10 0.809 0.749



10 0.0 50 0.811 0.749



10 0.0 100 0.806 0.749



10 0.0 500 0.791 0.749


...


120 100.0 50 0.804 0.749



120 100.0 100 0.79 0.749



120 100.0 500 0.782 0.749



120 100.0 1000 0.78 0.749
0.825 20 0.1 10


(0.825, 20, 0.1, 10)

In [153]:
item_vecs, user_vecs, model = train_model(product_train, 'als')
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

weighting matrix by bm25_weight
training model %s als



trained model '%s' in %s als 0.23572564125061035


(0.823, 0.749)

In [130]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [131]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items  
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    #print(purchased_ind)
    return item_lookup.loc[item_lookup.PRODUCT_DIM_KEY.isin(prod_codes)]


In [132]:
item_lookup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40596 entries, 0 to 117149
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PRODUCT_DIM_KEY  40596 non-null  int64 
 1   PRODUCT_NAME     40596 non-null  object
dtypes: int64(1), object(1)
memory usage: 951.5+ KB


In [133]:
customers_arr[:10]

array([31766, 31767, 31768, 31769, 31770, 31771, 31772, 31773, 31774,
       31775], dtype=int64)

In [160]:
get_items_purchased(33774, product_train, customers_arr, products_arr, item_lookup)

,PRODUCT_DIM_KEY,PRODUCT_NAME
42462,132979,"Dot Green Wine (Set Of 4) 8 In, 7 Oz"
49277,121173,Two Block Monogram Hand 14 x 21 Linen Embroide...
57224,109321,Four Crab Napkin Rings\n
62464,120564,Two Dragonfly 14 x 20 Linen Embroidered Guest ...


In [159]:
rec_items(33774, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,PRODUCT_DIM_KEY,PRODUCT_NAME
0,105050,Melrose 120 Round Tablecloth\n
1,121131,Two Oval Monogram Hand 14 x 21 Linen Embroider...
2,111357,Captiva Raspberry Wine Glass 8.25 in High 6 oz
3,350331,"Queen Anne's Lace 5"" x 7"" Frame with Decorativ..."
4,362216,White Orchid Tissue Box Holder
5,109541,Tropic Monogram 9 Round Side Dish
6,335798,Ginkgo Catch All
7,146762,Festival Specialty 15 X 90 Rect Table Runner (...
8,336144,Grapeleaf Snack Plate Gold
9,136893,Six Pino 27 x 27 Kitchen Towels


In [51]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.PRODUCT_NAME.loc[item_lookup.PRODUCT_DIM_KEY == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'PRODUCT_DIM_KEY': codes, 'PRODUCT_NAME': descriptions}) # Create a dataframe 
    return final_frame[['PRODUCT_DIM_KEY', 'PRODUCT_NAME']] # Switch order of columns around

In [161]:
get_items_purchased(37769, product_train, customers_arr, products_arr, item_lookup)

,PRODUCT_DIM_KEY,PRODUCT_NAME
26871,140775,Moroccan Amber Reed Diffuser 5.9 fl.oz/175ml
33704,134299,La Foret Ete Indien Set of Four 22 X 22 in Nap...
40282,145936,Festival Specialty 66 X 86 Rect Tablecloth (Cu...
58258,103688,Bellagio Table Linens 70 x 126 Oval Tablecloth\n
62894,101791,Twelve 18 x 18 Napkins


In [162]:
rec_items(37769, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,PRODUCT_DIM_KEY,PRODUCT_NAME
0,100833,Classico 70 x 162 Oval Linen Tablecloth
1,119851,Cardigan Firm King Pillow (26 Oz)
2,103756,Ginkgo Table Linens 70 x 90 Rect Linen Embroid...
3,154674,Custom Carlton Cotton Tablecloth
4,146194,Festival Specialty 66 X 140 Rect Tablecloth (C...
5,104097,Acanthus 104 Round Tablecloth
6,119855,Cardigan Medium King Pillow (22 Oz)
7,123863,Silhouette Table Knife 9.25 in Long
8,100432,Six Monogrammed 6 x 6 Cocktail Napkins
9,107669,Fiona King Pillow Protector 20 X 36 In
